In [ ]:


import numpy as np
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive')

!export SM_FRAMEWORK=tf.keras

!export SM_FRAMEWORK=tf.keras

# !pip install -U -q segmentation-models
# !pip install -q tensorflow==2.2.1
# !pip install -q keras==2.5
# import os
# os.environ["SM_FRAMEWORK"] = "tf.keras"

# from tensorflow import keras
# import segmentation_models as sm

%env SM_FRAMEWORK=tf.keras

import tensorflow as tf
import glob
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
import segmentation_models as sm

import glob
import cv2
import os
images_t = glob.glob("/content/drive/MyDrive/indianretinopathy/A.%20Segmentation/A. Segmentation/1. Original Images/a. Training Set/*.jpg")
images_t.sort()
images_v = glob.glob("/content/drive/MyDrive/indianretinopathy/A.%20Segmentation/A. Segmentation/1. Original Images/b. Testing Set/*.jpg")
images_v.sort()
masks_t = glob.glob("/content/drive/MyDrive/indianretinopathy/A.%20Segmentation/A. Segmentation/2. All Segmentation Groundtruths/a. Training Set/5. Optic Disc/*.tif")
masks_t.sort()
masks_v = glob.glob("/content/drive/MyDrive/indianretinopathy/A.%20Segmentation/A. Segmentation/2. All Segmentation Groundtruths/b. Testing Set/5. Optic Disc/*.tif")
masks_v.sort()
print(len(images_t))
print(len(images_v))
print(len(masks_t))
print(len(masks_v))

import os
image_t=[]
image_v=[]
mask_t=[]
mask_v=[]
for x in images_t:
  im=cv2.imread(x,cv2.IMREAD_COLOR)
  im=cv2.resize(im,(512,512))
  image_t.append(im)
for x in images_v:
  im=cv2.imread(x,cv2.IMREAD_COLOR)
  im=cv2.resize(im,(512,512))
  image_v.append(im)
for y in masks_t:
  ma=cv2.imread(y,cv2.IMREAD_GRAYSCALE)
  ma=cv2.resize(ma,(512,512),interpolation = cv2.INTER_NEAREST)
  plt.imshow(ma)
  mask_t.append(ma)
for y in masks_v:
  ma=cv2.imread(y,cv2.IMREAD_GRAYSCALE)
  ma=cv2.resize(ma,(512,512),interpolation = cv2.INTER_NEAREST)
  plt.imshow(ma)
  mask_v.append(ma)

print(len(image_t),len(mask_t))
img_array_t=np.array(image_t)
mask_array_t=np.array(mask_t)
mask_array_t=np.expand_dims(mask_array_t,axis=-1)
img_array_v=np.array(image_v)
mask_array_v=np.array(mask_v)
mask_array_v=np.expand_dims(mask_array_v,axis=-1)
mask_array_t=mask_array_t.astype(np.float32)
mask_array_v=mask_array_v.astype(np.float32)
print(img_array_t.shape)
print(mask_array_t.shape)
print((np.unique(mask_array_t)))
print(mask_array_t.shape)

a=np.where(mask_array_t==76)
mask_array_t[a]=1
print(np.unique(mask_array_t))
b=np.where(mask_array_v==76)
mask_array_v[b]=1
print(np.unique(mask_array_v))

import segmentation_models as sm
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from tensorflow.keras.utils import plot_model
sm.set_framework('tf.keras')
sm.framework()
BACKBONE = 'efficientnetb4'

preprocess_input = get_preprocessing(BACKBONE)
model = Unet(BACKBONE, input_shape=(512,512,3),encoder_weights='imagenet',activation='sigmoid')
# plot_model(model)


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='binary_crossentropy',metrics = ['binary_accuracy',tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), tf.keras.metrics.AUC(),sm.metrics.IOUScore(),sm.metrics.FScore()])
 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
mp=ModelCheckpoint('best_seg model.h5',save_best_only=True,monitor='val_iou_score')
lrp=ReduceLROnPlateau(monitor="iou_score", factor=0.1, patience=2)
es=EarlyStopping(patience=2,monitor='iou_score')
call=[lrp]
history=model.fit(img_array_t,
          mask_array_t,
          batch_size=1,
          epochs=10,
          validation_data=(img_array_v, mask_array_v),callbacks=call)

test_preds = model.predict(img_array_v)
preds_test_thresh = (test_preds >= 0.5).astype(np.uint8)
test_img = preds_test_thresh[5, :, :, 0]
plt.imshow(test_img, cmap='gray')

len(history.history['val_loss'])
total_epoch = len(history.history['val_loss'])

loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,total_epoch+1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

loss_train = history.history['binary_accuracy']
loss_val = history.history['val_binary_accuracy']
epochs = range(1,total_epoch+1)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

for i in range(0,len(img_array_v)):
    plt.figure(figsize=(15,15))
    plt.axis('Off')
    test_image = img_array_v[i, :, :, 0]
    plt.subplot(1,3,1)
    plt.imshow(test_image)
    plt.title('Input Image', fontsize=14)
    plt.axis('off')

    test_mask=mask_array_v[i,:,:]
    plt.subplot(1,3,2)
    plt.imshow(test_mask)
    plt.title('Ground Truth Mask')
    plt.axis('off')

    test_mask = preds_test_thresh[i, :, :, 0]
    plt.subplot(1,3,3)
    plt.imshow(test_mask, cmap='gray')
    plt.title('Pred Mask', fontsize=14)
    plt.axis('off')

sz = len(history.history['val_loss'])
print("Training Scores:")
print("Precision:",history.history['precision'][sz-1])
print("Recall:",history.history['recall'][sz-1])
print("iou_score:",history.history['iou_score'][sz-1])
print("f1-score:",history.history['f1-score'][sz-1])

print("")
print("Validation Scores:")
print("Precision:",history.history['val_precision'][sz-1])
print("Recall:",history.history['val_recall'][sz-1])
print("iou_score:",history.history['val_iou_score'][sz-1])
print("f1-score:",history.history['val_f1-score'][sz-1])

